In [13]:
import numpy as np

from goForwardGame import vectorizedGoForwardGamesTest
from ga_neural_net import ANN

vectorizedGame = vectorizedGoForwardGamesTest()

In [14]:
vectorizedGame.reset()

In [15]:
players = []
for _ in range(4):
    players.append(ANN(layers_unit=[732, 1024, 1024, 1]))

gen = 1

In [16]:
EPOCH_GAME = 4 # Number of games before select best to crossover
EPOCH = 10

logfile = open('log.csv', 'a')

for _ in range(EPOCH):
    scoreboard = np.zeros(4)
    for game_index in range(EPOCH_GAME):
        vectorizedGame.reset()
        is_done = False
        game_steps_history = []

        logfile.write(f'{gen},{game_index+1},')
        print(f'gen {gen}, game {game_index+1}')
        # Play a game
        while not is_done:
            _, string_cards, actions, number_actions = vectorizedGame.info()
            current_player, state, action_space = vectorizedGame.getCurrentState()
            step = min(int(players[current_player-1].get_step(state[0]) * len(number_actions)), len(number_actions)-1)
            rewards, is_done, end_info = vectorizedGame.step(number_actions[step])
            game_steps_history.append(number_actions[step])

        scoreboard += rewards
        logfile.write(f'"{str(game_steps_history)}","{rewards}"\n')
    print('After', EPOCH_GAME, 'game scoreboard:', scoreboard/EPOCH_GAME)
    
    rank = scoreboard.argsort()[::-1]
    players[rank[2]] = players[rank[0]].fuck_with(players[rank[1]], 0.5)
    players[rank[3]] = players[rank[0]].fuck_with(players[rank[1]], 0.1)
    gen+=1

logfile.close()

gen 1, game 1
gen 1, game 2
gen 1, game 3
gen 1, game 4
After 4 game scoreboard: [ 2.   11.5  -9.25 -4.25]
gen 2, game 1
gen 2, game 2
gen 2, game 3
gen 2, game 4
After 4 game scoreboard: [ 14.   11.5 -13.  -12.5]
gen 3, game 1
gen 3, game 2
gen 3, game 3
gen 3, game 4
After 4 game scoreboard: [ 29.    -4.25 -12.25 -12.5 ]
gen 4, game 1
gen 4, game 2
gen 4, game 3
gen 4, game 4
After 4 game scoreboard: [ -3.   26.  -12.5 -10.5]
gen 5, game 1
gen 5, game 2
gen 5, game 3
gen 5, game 4
After 4 game scoreboard: [ 12.25  11.5  -12.25 -11.5 ]
gen 6, game 1
gen 6, game 2
gen 6, game 3
gen 6, game 4
After 4 game scoreboard: [ 11.25  13.   -11.25 -13.  ]
gen 7, game 1
gen 7, game 2
gen 7, game 3
gen 7, game 4
After 4 game scoreboard: [ -1.75  27.   -12.25 -13.  ]
gen 8, game 1
gen 8, game 2
gen 8, game 3
gen 8, game 4
After 4 game scoreboard: [  3.75  22.25 -13.   -13.  ]
gen 9, game 1
gen 9, game 2
gen 9, game 3
gen 9, game 4
After 4 game scoreboard: [ 12.25  13.   -12.25 -13.  ]
gen 10, game 

In [17]:
# Export model
players[0].export_model('model.json')

In [18]:
# Load model from saved file
model = ANN.load_model('model.json')